d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Cross-Validation Lab

**Objective**: *Assess your ability to apply cross-validated hyperparameter tuning to a model.*

In this lab, you will apply what you've learned in this lesson. When complete, please use the answers to the exercises to answer questions in the following quiz within Coursera.

In [0]:
%run "../../Includes/Classroom-Setup"

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Out[2]: DataFrame[]

res1: Boolean = false

res2: Boolean = false

res3: Boolean = false

-sandbox

## Exercise 1

In this exercise, you will create an enhanced user-level table to try to better predict whether or not each user takes at least *8,000* steps in a day. For this exercise, assume we only have access to heart rate information.

Fill in the blanks in the below cell to create the `adsda.ht_user_metrics_cv_lab` table.

Note that this lab is focused on predicting whether users take 8,000 steps per day rather than 10,000 steps per day.

In [0]:
%sql
-- TODO
CREATE OR REPLACE TABLE adsda.ht_user_metrics_cv_lab
USING DELTA LOCATION "/adsda/ht-user-metrics-cv-lab" AS (
  SELECT min(resting_heartrate) AS min_resting_heartrate,
         avg(resting_heartrate) AS avg_resting_heartrate,
         max(resting_heartrate) AS max_resting_heartrate,
         max(resting_heartrate) - min(resting_heartrate) AS resting_heartrate_change,
         min(active_heartrate) AS min_active_heartrate,
         avg(active_heartrate) AS avg_active_heartrate,
         max(active_heartrate) AS max_active_heartrate,
         max(active_heartrate) - min(active_heartrate) AS active_heartrate_change,
         CASE WHEN avg(steps) >= 8000 THEN 1 ELSE 0 END AS steps_8000
  FROM adsda.ht_daily_metrics
  GROUP BY device_id
)

num_affected_rows,num_inserted_rows


-sandbox
**Coursera Quiz:** How many users in `adsda.ht_user_metrics_cv_lab` take, on average, 8,000 steps per day?

**Hint:** Refer back to the previous lab for guidance on how to answer this question.

In [0]:
%sql
-- TODO
SELECT steps_8000, count(*) FROM adsda.ht_user_metrics_cv_lab GROUP BY steps_8000

steps_8000,count(1)
1,1924
0,1076


-sandbox
## Exercise 2

In this exercise, you will split your data into a cross-validation set (`cross_val_df`) and test set (`test_df`).

Fill in the blanks below to split your data.

**Hint:** Refer to the previous demo for guidance.

In [0]:
# TODO
from sklearn.model_selection import train_test_split

ht_user_metrics_pd_df = spark.table("adsda.ht_user_metrics_cv_lab").toPandas()

cross_val_df, test_df = train_test_split(ht_user_metrics_pd_df, train_size=0.80, test_size=0.20, random_state=42)

**Coursera Quiz:** How many rows are in the `cross_val_df` DataFrame?

Fill in the blanks below to answer the question.

In [0]:
# TODO
cross_val_df.shape

Out[7]: (2400, 9)

## Exercise 3

In this exercise, you will prepare your random forest classifier.

Fill in the blanks below to complete the task.

In [0]:
# TODO
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)

## Exercise 4

In this exercise, you will create a hyperparameter grid to use during the grid search process.

Use the following hyperparameter values:

1. `max_depth`: 5, 8, 20
1. `n_estimators`: 25, 50, 100
1. `min_samples_split`: 2, 4
1. `max_features`: 3, 4
1. `max_samples`: 0.6, 0.8

Fill in the blanks below to create the grid.

In [0]:
# TODO
parameter_grid = {
  "max_depth": [5, 8, 20],
  "n_estimators": [25, 50, 100],
  "min_samples_split": [2, 4],
  "max_features": [3, 4],
  "max_samples": [0.6, 0.8]
}

-sandbox
**Coursera Quiz**: How many total unique combinations of hyperparameters are there in `parameter_grid`?

Use the below empty cell to determine the answer to the above question.

**Hint:** Refer to the previous lesson's lab for guidance.

In [0]:
# TODO
len(parameter_grid["max_depth"]) * len(parameter_grid["n_estimators"]) * len(parameter_grid["min_samples_split"]) * len(parameter_grid["max_features"]) * len(parameter_grid["max_samples"])

Out[10]: 72

## Exercise 5

In this exercise, you will create the cross-validated grid-search object that you will use to optimize your hyperparameter values while using cross-validation.

Fill in the blanks below to create the object.

**NOTE:** Please use 4-fold cross-validation.

In [0]:
# TODO
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=rfc, cv=4, param_grid=parameter_grid)

## Exercise 6

In this exercise, you will fit the grid search process.

Fill in the blanks below to perform the grid search process.

In [0]:
# TODO
grid_search.fit(cross_val_df.drop("steps_8000", axis=1), cross_val_df["steps_8000"])

Out[13]: GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=42),
 param_grid={'max_depth': [5, 8, 20], 'max_features': [3, 4],
 'max_samples': [0.6, 0.8], 'min_samples_split': [2, 4],
 'n_estimators': [25, 50, 100]})

-sandbox
**Coursera Quiz**: How many unique models are being trained by the cross-validated grid search process?

* 4
* 289
* 21
* 288

**Hint:** Consider the number of unique feature combinations, the number of cross-validation folds and the final retraining of the model on the entire cross-validation set.

## Exercise 7

In this exercise, you will return a Pandas DataFrame of the `grid_search` results.

Fill in the blanks below to return the DataFrame.

In [0]:
# TODO
import pandas as pd
pd.DataFrame(grid_search.cv_results_)

Out[14]:

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_max_samples,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.084545,0.007498,0.009324,0.000683,5,3,0.6,2,25,"{'max_depth': 5, 'max_features': 3, 'max_sampl...",0.891667,0.876667,0.885000,0.891667,0.886250,0.006166,41
1,0.149119,0.004502,0.012663,0.000345,5,3,0.6,2,50,"{'max_depth': 5, 'max_features': 3, 'max_sampl...",0.896667,0.888333,0.885000,0.893333,0.890833,0.004488,12
2,0.293689,0.003548,0.022655,0.002211,5,3,0.6,2,100,"{'max_depth': 5, 'max_features': 3, 'max_sampl...",0.896667,0.890000,0.890000,0.895000,0.892917,0.002976,2
3,0.078519,0.001718,0.009250,0.000727,5,3,0.6,4,25,"{'max_depth': 5, 'max_features': 3, 'max_sampl...",0.891667,0.886667,0.886667,0.891667,0.889167,0.002500,21
4,0.151240,0.005908,0.012924,0.000513,5,3,0.6,4,50,"{'max_depth': 5, 'max_features': 3, 'max_sampl...",0.895000,0.885000,0.888333,0.893333,0.890417,0.003975,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.217310,0.003316,0.013659,0.000398,20,4,0.8,2,50,"{'max_depth': 20, 'max_features': 4, 'max_samp...",0.881667,0.868333,0.868333,0.873333,0.872917,0.005449,72
68,0.430906,0.009229,0.025106,0.002210,20,4,0.8,2,100,"{'max_depth': 20, 'max_features': 4, 'max_samp...",0.895000,0.871667,0.868333,0.881667,0.879167,0.010375,67
69,0.112343,0.004744,0.008975,0.000109,20,4,0.8,4,25,"{'max_depth': 20, 'max_features': 4, 'max_samp...",0.898333,0.868333,0.878333,0.886667,0.882917,0.011016,54
70,0.214988,0.002751,0.013387,0.000104,20,4,0.8,4,50,"{'max_depth': 20, 'max_features': 4, 'max_samp...",0.890000,0.870000,0.875000,0.886667,0.880417,0.008197,60


## Exercise 8

In this exercise, you will identify the optimal hyperparameter values.

Fill in the blanks below to indentify the optimal hyperparameter values.

In [0]:
# TODO
grid_search.best_params_

Out[15]: {'max_depth': 5,
 'max_features': 4,
 'max_samples': 0.8,
 'min_samples_split': 2,
 'n_estimators': 25}

**Coursera Quiz:** What is the optimal hyperparameter value for `max_depth` according to the cross-validated grid search process?

## Exercise 9

In this exercise, you will identify the test accuracy achieved by the final, refit model.

Fill in the blanks below to identify the test accuracy.

In [0]:
# TODO
from sklearn.metrics import accuracy_score

accuracy_score(
  test_df["steps_8000"], 
  grid_search.predict(test_df.drop("steps_8000", axis=1))
)

Out[18]: 0.895

**Coursera Quiz:** What is the test set accuracy?

Congratulations! That concludes our lesson on cross-validated hyperparameter optimization and our course!

Be sure to submit your quiz answers to Coursera to fully complete the course!

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>